# VGGNet

重复网络结构的构建



In [1]:
from mxnet.gluon import nn
def vgg_block3(num_convs,channels):
    out = nn.Sequential()
    for _ in range(num_convs):
        out.add(nn.Conv2D(channels,kernel_size=3,padding=1,activation='relu'))
    out.add(nn.MaxPool2D(pool_size=2,strides=2))
    return out
def vgg_block3_1(num_convs,channels):
    out = nn.Sequential()
    for _ in range(num_convs):
        out.add(nn.Conv2D(channels,kernel_size=3,padding=1,activation='relu'))
    out.add(nn.Conv2D(channels,kernel_size=1,padding=1,activation='relu'),
           nn.MaxPool2D(pool_size=2,strides=2))
    return out
def vgg_fc(num_outputs):
    out = nn.Sequential()
    out.add(nn.Flatten(),
            nn.Dense(4096,activation='relu'),
            nn.Dropout(.5),
            nn.Dense(4096,activation='relu'),
            nn.Dropout(.5),
            nn.Dense(num_outputs)
           )
    return out

In [2]:
def get_vgg(architecture,num_outputs):
    net = nn.Sequential()
    with net.name_scope():
        for num_convs,channels in architecture:
            net.add(vgg_block3(num_convs,channels))
        net.add(vgg_fc(num_outputs))
    return net


获取数据，初始化网络，设置loss和solver

In [ ]:
import sys
sys.path.append('..')
import utils
from mxnet import init
from mxnet import gluon

batch_size = 128
resize=128
train_data, test_data = utils.load_data_fashion_mnist(batch_size,resize=resize)
ctx = utils.try_gpu()

num_outputs = 10
architecture = ((1,64),(1,128),(2,256),(2,512),(2,512))
net=get_vgg(architecture,num_outputs)
net.initialize(ctx=ctx,init=init.Xavier())

loss = gluon.loss.SoftmaxCrossEntropyLoss()
#train(train_data, test_data, net, loss, trainer, ctx, num_epochs, print_batches=None):
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.05})
utils.train(train_data,test_data,net,loss,trainer,ctx,num_epochs=5)

('Start training on ', gpu(0))
Epoch 0. Loss: 1.011, Train acc 0.62, Test acc 0.81, Time 93.4 sec
